This notebook makes a time line of the treatment of patients based on the corresponding protocol. The figures ofeach patient are then ensambled in Additiona file 1 Figure S3

In [ ]:
import pandas as pd
import numpy as np
import glob
from collections import OrderedDict

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatch
import seaborn as sns

from aux_data_in_pyvar import config_rcparams

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [ ]:
config_rcparams()
plt.rcParams['font.size'] = 14

In [ ]:
# output path for the figures
out_path = ""

# clinical data
clinical = pd.read_csv("", sep='\t') # Additional file 1 Table S1
clinical.tail()

In [ ]:
# we don't know the protocol of PAT16. Let's assume its LAL-AR/2011 
index = clinical[clinical['Patient_id'] == 'PAT16'].index[0]
clinical.set_value(index, 'Treatment_protocol' , 'LAL-AR/2011')
clinical.set_value(index, 'Maintenance' , 'Yes')

#### protocol LAL-AR/2011

In [ ]:
timing = OrderedDict({'P':5,
    'I-1':28,
         'I-2':7,
         'C-1':7,
         'C-2':7,
         'C-3':7,
         'C-1d':7,
         'C-2d':7,
         'C-3d':7,
         'M':227})

cum_sum = OrderedDict({'Induction1_14_blasts':np.nan, 
       'Induction1_35_blasts':np.nan,
       'Induction2_blasts':np.nan,
       'Consolidation1_blasts':np.nan, 
       'Consolidation3_blasts':np.nan})

timing_colors = {'P':'#f4d682',
                 'I-1':"#ffa589",
                 'I-2':"#ff7575",
                 'C-1':"#59D8E6",
                 'C-2':"#00bbec",
                 'C-3':"#3498DB",
                 'C-1d':"#eedaf2ff",
                 'C-2d':"#ddb5e4ff",
                 'C-3d':"#cd91d7ff",
                 'M':"#1F8A70"}


In [ ]:
for num,pat in enumerate(clinical[clinical['Treatment_protocol'] == 'LAL-AR/2011']['Patient_id'].unique()):
    print(pat)
    clinical_pat = clinical[clinical['Patient_id'] == pat].reset_index()
    
    #make figure
    fig,ax = plt.subplots(figsize=(22,5))
    lenght =  365+130
    ax.set_ylim(-8,2)
    ax.set_xlim(-8,lenght)

    # create timeline
    ax.hlines(xmin=-5,xmax=lenght,y=-0.15,lw=2,color="black",alpha=1.)

    # create periods of treatmet
    i=0
    for k in timing.keys():
        if i == 0:
            # period
            rect = mpatch.Rectangle(xy=(0,0),width=timing[k],height=0.5,color=timing_colors[k],alpha=1., zorder=2)
            ax.annotate(s=k,xy=(timing[k]/2, 0.23),fontweight='bold',ha='center',va='center')
            ax.add_patch(rect)

            #treatment
            ax.vlines(ymin=-3.8,ymax=0, x=i+(timing[k]/2),lw=1,color="black",alpha=0.7,linestyles='dotted')
            ax.annotate(s="PDN\nITT",xy=(i+(timing[k]/2),-4.5), va='top', ha='right')
            i = i+timing[k]
        else:
            # period
            if k == 'I-1':
                rect = mpatch.Rectangle(xy=(i,0),width=timing[k],height=0.5,color=timing_colors[k],alpha=1., zorder=2)
                ax.annotate(s=k,xy=(i+(timing[k]/2),0.23),ha='center',va='center',fontweight='bold')
                ax.add_patch(rect)

                #treatment
                ax.vlines(ymin=-3.8,ymax=0, x=i+(timing[k]/2),lw=1,color="black",alpha=0.7,linestyles='dotted')
                ax.annotate(s="VCR\nDNR\nASA\nITT",xy=(i+(timing[k]/2),-4.5), 
                            va='top', ha='center')
                i = i+timing[k]+7 #one week interval break
                cum_sum['Induction1_14_blasts'] = 14+5
                cum_sum['Induction1_35_blasts'] = 35+5
            elif k == 'I-2':
                # evaluate
                if pd.isnull(clinical_pat.loc[0, 'Induction1_35_blasts']) == False:
                    measure_i35 = clinical_pat.loc[0, 'Induction1_35_blasts']
                    measure_i35 = measure_i35.replace("<", "")
                    measure_i35 = measure_i35.replace(">", "")
                    if float(measure_i35) > 0.1:
                        #period
                        rect = mpatch.Rectangle(xy=(i,0),width=timing[k],height=0.5,color=timing_colors[k],alpha=1., zorder=2)
                        ax.annotate(s=k,xy=(i+(timing[k]/2),0.23),ha='center',va='center',fontweight='bold')
                        ax.add_patch(rect)

                        # treatment
                        ax.vlines(ymin=-3.8,ymax=0, x=i+(timing[k]/2),lw=1,color="black",alpha=0.7,linestyles='dotted')
                        ax.annotate(s="FLAG-\nIDA\nITT",xy=(i+(timing[k]/2),-4.5), 
                                    va='top', ha='center')
                        
                        cum_sum['Induction2_blasts'] = i+timing[k]+3
                        i = i+timing[k]+14 #one week interval break
                        
                elif pd.isnull(clinical_pat.loc[0, 'Induction2_blasts']) == False:
                    #period
                    rect = mpatch.Rectangle(xy=(i,0),width=timing[k],height=0.5,color=timing_colors[k],alpha=1., zorder=2)
                    ax.annotate(s=k,xy=(i+(timing[k]/2),0.23),ha='center',va='center',fontweight='bold')
                    ax.add_patch(rect)

                    # treatment
                    ax.vlines(ymin=-3.8,ymax=0, x=i+(timing[k]/2),lw=1,color="black",alpha=0.7,linestyles='dotted')
                    ax.annotate(s="FLAG-\nIDA\nITT",xy=(i+(timing[k]/2),-4.5), 
                                va='top', ha='center')
                    cum_sum['Induction2_blasts'] = i+timing[k]+3
                    i = i+timing[k]+14 # interval break
                    
                else:
                    pass
            elif k.startswith('C-'):
                    if not k.endswith('d'):
                        # periods
                        rect = mpatch.Rectangle(xy=(i,0),width=timing[k],height=0.5,color=timing_colors[k],alpha=1., zorder=2)
                        ax.annotate(s=k,xy=(i+(timing[k]/2),0.23),ha='center',va='center',fontweight='bold')
                        ax.add_patch(rect)
                        # treatments
                        if k == 'C-1':
                            ax.vlines(ymin=-3.8,ymax=0, x=(i)+(timing[k]/2),lw=1,color="black",alpha=0.7,linestyles='dotted')
                            ax.annotate(s="DXM\nVCR\nMTX\nASA\nITT",xy=(i+(timing[k]/2),-4.5),ha='center',va='top')
                            cum_sum['Consolidation1_blasts'] = i+timing[k]+3
                            i = i+timing[k]+21 # interval break
                            
                        elif k == 'C-3':
                            ax.vlines(ymin=-3.8,ymax=0, x=(i)+(timing[k]/2),lw=1,color="black",alpha=0.7,linestyles='dotted')
                            ax.annotate(s="DXM\nVCR\nMTX\nASA\nITT",xy=(i+(timing[k]/2),-4.5),ha='center',va='top')
                            cum_sum['Consolidation3_blasts'] = i+timing[k]+3
                            i = i+timing[k]+30 # interval break
                            
                        else:
                            ax.vlines(ymin=-3.8,ymax=0, x=(i)+(timing[k]/2),lw=1,color="black",alpha=0.7,linestyles='dotted')
                            ax.annotate(s="DXM\nARA-C\nASA\nITT",xy=(i+(timing[k]/2),-4.5), 
                                        ha='center',va='top')
                            i = i+timing[k]+21 # interval break
                    else:
                        if pd.isnull(clinical_pat.loc[0, 'Consolidation3_blasts']) == False:
                            measure_c3 = clinical_pat.loc[0, 'Consolidation3_blasts']
                            measure_c3 = measure_i35.replace("<", "")
                            measure_c3 = measure_i35.replace(">", "")
                            print(measure_c3)
                            if float(measure_c3) < 0.01:
                                # periods
                                rect = mpatch.Rectangle(xy=(i,0),width=timing[k],height=0.5,color=timing_colors[k],alpha=1., zorder=2)
                                ax.annotate(s=k,xy=(i+(timing[k]/2),0.23),ha='center',va='center',fontweight='bold')
                                ax.add_patch(rect)
                                # treatments
                                if k == 'C-1d':
                                    ax.vlines(ymin=-3.8,ymax=0, x=(i)+(timing[k]/2),lw=1,color="black",alpha=0.7,linestyles='dotted')
                                    ax.annotate(s="DXM\nVCR\nMTX\nASA\nITT",xy=(i+(timing[k]/2),-4.5),ha='center',va='top')
                                    i = i+timing[k]+21 # interval break
                                elif k == 'C-3d':
                                    ax.vlines(ymin=-3.8,ymax=0, x=(i)+(timing[k]/2),lw=1,color="black",alpha=0.7,linestyles='dotted')
                                    ax.annotate(s="DXM\nVCR\nMTX\nASA\nITT",xy=(i+(timing[k]/2),-4.5),ha='center',va='top')
                                    i = i+timing[k]+21 # interval break
                                else:
                                    ax.vlines(ymin=-3.8,ymax=0, x=(i)+(timing[k]/2),lw=1,color="black",alpha=0.7,linestyles='dotted')
                                    ax.annotate(s="DXM\nARA-C\nASA\nITT",xy=(i+(timing[k]/2),-4.5), 
                                                ha='center',va='top')
                                    i = i+timing[k]+21 # interval break
                            else:
                                if clinical_pat.loc[0,'Transplant'] == 'Yes':

                                    # treatment
                                    ax.vlines(ymin=-3.8,ymax=0, x=i+(timing[k]/4),lw=1,color="black",alpha=0.7,linestyles='dotted')
                                    ax.annotate(s="Alo-HSCT",xy=(i+(timing[k]/4),-4.5), ha='center',va='top')
            else:
                if clinical_pat.loc[0,'Maintenance'] == 'Yes':
                
                    # period
                    rect = mpatch.Rectangle(xy=(i,0),width=360-i,height=0.5,color=timing_colors[k],alpha=1., zorder=2)
                    ax.annotate(s=k,xy=(i+((360-i)/2),0.23),ha='center',va='center',fontweight='bold')
                    ax.add_patch(rect)

                    # treatment
                    ax.vlines(ymin=-3.8,ymax=0, x=i+((360-i)/2),lw=1,color="black",alpha=0.7,linestyles='dotted')
                    ax.annotate(s="MTX\nMP\nVCR\nPDN",xy=(i+((360-i)/2),-4.5), 
                                ha='center',va='top')

                    i = i+timing[k]

                else:
                    if clinical_pat.loc[0,'Transplant'] == 'Yes':

                        # treatment
                        ax.vlines(ymin=-3.8,ymax=0, x=i+(timing[k]/4),lw=1,color="black",alpha=0.7,linestyles='dotted')
                        ax.annotate(s="Alo-HSCT",xy=(i+(timing[k]/4),-4.5), ha='center',va='top')

    # Diagnosis date and blasts

    ax.scatter(x=-0.5, y=1, marker='o', color='black') 
    ax.vlines(ymin=0,ymax=1, x=-0.5,lw=1,color="black",alpha=1.)
    ax.text(s="Sample at diagnosis: \n {} %blasts".format(clinical_pat.loc[0,'Primary_sample_blasts']), 
            x=-0.5,y=1.2,rotation=45, va='bottom')

    ax.scatter(x=-0.5, y=-0.5, marker='o', color='black') 
    ax.vlines(ymin=-0.5,ymax=0, x=-0.5,lw=1,color="black",alpha=1.)
#     ax.text(s="Diagnosis\n{}".format(str(clinical_pat.loc[0,'Primary_biopsy_date'])), 
    ax.text(s="Diagnosis\n1", 
            x=-0.5, y=-0.5,rotation=45, va='top', ha='right', 
           bbox=dict(fc="#f0f0f0", ec='#5b6b6a', pad=1, boxstyle="round,pad=0.1"))

    # Remission date and blasts
    
    if pd.isnull(clinical_pat.loc[0, 'days_between_pry_rem']) == False:
        ax.scatter(x=clinical_pat.loc[0,'days_between_pry_rem'], y=-0.5, marker='o', color='black') 
        ax.vlines(ymin=-0.5,ymax=0, x=clinical_pat.loc[0,'days_between_pry_rem'],lw=1,color="black",alpha=1.)
    #    ax.text(s="Remission\n{}".format(str(clinical_pat.loc[0,'Remission_biopsy_date'])),
        ax.text(s="Remission\n{}".format(str(int(clinical_pat.loc[0,'days_between_pry_rem']))),
                x=clinical_pat.loc[0,'days_between_pry_rem'],y=-0.5,rotation=45, 
                va='top', ha='right', 
               bbox=dict(fc="#f0f0f0", ec='#5b6b6a', pad=1, boxstyle="round,pad=0.1"))

        ax.scatter(x=clinical_pat.loc[0,'days_between_pry_rem'], y=1.2, marker='o', color='black') 
        ax.vlines(ymin=0,ymax=1.2, x=clinical_pat.loc[0,'days_between_pry_rem'],lw=1,color="black",alpha=1.)
        ax.text(s="Sample at remission \n {} %blasts".format(clinical_pat.loc[0,'Remission_sample_blasts']), 
                x=clinical_pat.loc[0,'days_between_pry_rem'],y=1.2,rotation=45, va='bottom')

    # Relapse date and blasts
    
    if pd.isnull(clinical_pat.loc[0,'days_between_pry_rel']) == False:
        ax.scatter(x=clinical_pat.loc[0,'days_between_pry_rel'], y=-0.5, marker='o', color='black') 
        ax.vlines(ymin=-0.5,ymax=0, x=clinical_pat.loc[0,'days_between_pry_rel'],lw=1,color="black",alpha=1.)
#        ax.text(s="Relapse\n{}".format(str(clinical_pat.loc[0,'Relapse_biopsy_date'])), 
        ax.text(s="Relapse\n{}".format(str(int(clinical_pat.loc[0,'days_between_pry_rel']))), 
                x=clinical_pat.loc[0,'days_between_pry_rel'],y=-0.5,rotation=45, 
                va='top', ha='right', 
                bbox=dict(fc="#f0f0f0", ec='#5b6b6a', pad=1, boxstyle="round,pad=0.1"))

        ax.scatter(x=clinical_pat.loc[0,'days_between_pry_rel'], y=1.2, marker='o', color='black') 
        ax.vlines(ymin=0,ymax=1.2, x=clinical_pat.loc[0,'days_between_pry_rel'],lw=1,color="black",alpha=1.)
        ax.text(s="Sample at relapse \n {} %blasts".format(clinical_pat.loc[0, "Relapse_sample_blasts"]), 
            x=clinical_pat.loc[0,'days_between_pry_rel'],y=1.2,rotation=45, va='bottom')

    # One year limit
    ax.scatter(x=365, y=-0.5, marker='o', color='black') 
    ax.vlines(ymin=-0.5,ymax=0, x=365,lw=1,color="black",alpha=1.)
    ax.text(s="One year\nfrom diagnosis", x=365,y=-0.5,rotation=45, va='top', ha='right')

    # other blast information

    for measure, days in cum_sum.items():
        if pd.isnull(clinical_pat.loc[0,measure]) == False:
            print(clinical_pat.loc[0,measure])
            ax.scatter(x=cum_sum[measure], y=1.2, marker='o', color='black') 
            ax.vlines(ymin=0,ymax=1.2, x=cum_sum[measure],lw=1,color="black",alpha=1.)
            ax.text(s="{} %blasts".format(clinical_pat.loc[0, measure]), 
                x=cum_sum[measure],y=1.2,rotation=45, va='bottom', ha='left')

    if clinical_pat.loc[0, 'Refractory'] == 'Yes':
        re_type = 'RFA'
    else:
        re_type = 'RC'
    ax.set_yticks([2,0,-2,-4,-6])
    ax.set_yticklabels(('',"{}\n{} {} yrs {}".format(pat, clinical_pat.loc[0,'Primary_immunoclassification'], 
                                        clinical_pat.loc[0,'Primary_diagnosis_age'], re_type),'',''),
                       fontweight='bold', ha='center')

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)

    plt.tick_params(
        axis='both',          
        which='both',      
        bottom=False,      
        top=False,
        right=False,
        left=False,
        labelbottom=False,
        pad=60)

    fig.savefig(os.path.join(out_path, "clinical_{}.svg").format(pat), dpi=300,  bbox_inches='tight')
    plt.show()

#### Protocols LAL-AR/2003 and LAL-07OLD

In [ ]:
#protocol = 'LAL-07OLD'
protocol ='ALL-HR-03'

cum_sum = OrderedDict({'Induction1_14_blasts':np.nan, 
       'Induction1_35_blasts':np.nan,
       'Induction2_blasts':np.nan,
       'Consolidation1_blasts':np.nan, 
       'Consolidation3_blasts':np.nan})

timing_colors = {'P':'#f4d682',
                 'I-1':"#ffa589",
                 'I-2':"#ff7575",
                 'C-1':"#59D8E6",
                 'C-2':"#00bbec",
                 'C-3':"#3498DB",
                 'C-1d':"#eedaf2ff",
                 'C-2d':"#ddb5e4ff",
                 'C-3d':"#cd91d7ff",
                 'M':"#1F8A70"}

if protocol == 'LAL-07OLD':
   
    ## LAL-07OLD
    timing = {'P':5,
             'I-1':14,
             'I-2':14,
             'C-1':7,
             'C-2':7,
             'C-3':7,
             'C-1d':7,
             'C-2d':7,
             'C-3d':7,
             'M':155+200}

    timing_treatment = {'P':"PRD\nITT",
                     'I-1':"VCR\nIDA\nDXM\nITT",
                     'I-2':"CFM\nARA-C",
                     'C-1':"MTX\nL-ASA",
                     'C-2':"ARA-C",
                     'C-3':"MTX\nL-ASA",
                     'C-1d':"ARA-C",
                     'C-2d':"MTX\nL-ASA",
                     'C-3d':"ARA-C",
                     'M':"MTX\nMP\nVCR\nPDN"}
    
elif protocol == 'ALL-HR-03':

    ## AR2003
    timing = {'P':5,
             'I-1':30,
             'C-1':14,
             'C-2':14,
             'C-3':14,
             'C-1d':14,
             'C-2d':14,
             'C-3d':14,
             'M':551}

    timing_treatment = {'P':"PRD\nITT",
                     'I-1':"VCR\nDNR\nPDR\nITT",
                     'C-1':"VCR\nDXM\nMTX\nARA-C\nL-ASA\nMP",
                     'C-2':"VCR\nDXM\nMTX\nCPM-C\nL-ASA\nMXN",
                     'C-3':"DXM\nARA-C\nTNP\nL-ASA",
                     'C-1d':"VCR\nDXM\nMTX\nARA-C\nL-ASA\nMP",
                     'C-2d':"VCR\nDXM\nMTX\nCPM-C\nL-ASA\nMXN",
                     'C-3d':"DXM\nARA-C\nTNP\nL-ASA",
                     'M':"MTX\nMP"}

else:
    pass

In [ ]:
for num,pat in enumerate(clinical[clinical['Treatment_protocol'] == protocol]['Patient_id'].unique()):
    print(pat)
    clinical_pat = clinical[clinical['Patient_id'] == pat].reset_index()
    
    #make figure
    fig,ax = plt.subplots(figsize=(22,5))
    lenght =  365+180
    ax.set_ylim(-8,2)
    ax.set_xlim(-8,lenght)

    # create timeline
    ax.hlines(xmin=-5,xmax=lenght,y=-0.15,lw=2,color="black",alpha=1.)

    # create periods of treatmet
    i=0
    for k in timing.keys():
        if i == 0:
            # period
            rect = mpatch.Rectangle(xy=(0,0),width=timing[k],height=0.5,color=timing_colors[k],alpha=1., zorder=2)
            ax.annotate(s=k,xy=(timing[k]/2, 0.23),fontweight='bold',ha='center',va='center')
            ax.add_patch(rect)

            #treatment
            ax.vlines(ymin=-3.8,ymax=0, x=i+(timing[k]/2),lw=1,color="black",alpha=0.7,linestyles='dotted')
            ax.annotate(s=timing_treatment[k],xy=(i+(timing[k]/2),-4.5), va='top', ha='right')
            i = i+timing[k]
        else:
            # period
            if k == 'I-1':
                rect = mpatch.Rectangle(xy=(i,0),width=timing[k],height=0.5,color=timing_colors[k],alpha=1., zorder=2)
                ax.annotate(s=k,xy=(i+(timing[k]/2),0.23),ha='center',va='center',fontweight='bold')
                ax.add_patch(rect)

                #treatment
                ax.vlines(ymin=-3.8,ymax=0, x=i+(timing[k]/2),lw=1,color="black",alpha=0.7,linestyles='dotted')
                ax.annotate(s=timing_treatment[k],xy=(i+(timing[k]/2),-4.5), 
                            va='top', ha='center')
                i = i+timing[k]+7 #one week interval break
                cum_sum['Induction1_14_blasts'] = 14+5
                cum_sum['Induction1_35_blasts'] = 35+5
            elif k == 'I-2':
                #period
                rect = mpatch.Rectangle(xy=(i,0),width=timing[k],height=0.5,color=timing_colors[k],alpha=1., zorder=2)
                ax.annotate(s=k,xy=(i+(timing[k]/2),0.23),ha='center',va='center',fontweight='bold')
                ax.add_patch(rect)

                # treatment
                ax.vlines(ymin=-3.8,ymax=0, x=i+(timing[k]/2),lw=1,color="black",alpha=0.7,linestyles='dotted')
                ax.annotate(s=timing_treatment[k],xy=(i+(timing[k]/2),-4.5), 
                            va='top', ha='center')

                cum_sum['Induction2_blasts'] = i+timing[k]+3
                i = i+timing[k]+14 #one week interval break
                        
            elif k.startswith('C-'):
                    if not k.endswith('d'):
                        # periods
                        rect = mpatch.Rectangle(xy=(i,0),width=timing[k],height=0.5,color=timing_colors[k],alpha=1., zorder=2)
                        ax.annotate(s=k,xy=(i+(timing[k]/2),0.23),ha='center',va='center',fontweight='bold')
                        ax.add_patch(rect)
                        # treatments
                        if k == 'C-1':
                            ax.vlines(ymin=-3.8,ymax=0, x=(i)+(timing[k]/2),lw=1,color="black",alpha=0.7,linestyles='dotted')
                            ax.annotate(s=timing_treatment[k],xy=(i+(timing[k]/2),-4.5),ha='center',va='top')
                            cum_sum['Consolidation1_blasts'] = i+timing[k]+3
                            i = i+timing[k]+21 # interval break
                            
                        elif k == 'C-3':
                            ax.vlines(ymin=-3.8,ymax=0, x=(i)+(timing[k]/2),lw=1,color="black",alpha=0.7,linestyles='dotted')
                            ax.annotate(s=timing_treatment[k],xy=(i+(timing[k]/2),-4.5),ha='center',va='top')
                            cum_sum['Consolidation3_blasts'] = i+timing[k]+3
                            i = i+timing[k]+30 # interval break
                            
                        else:
                            ax.vlines(ymin=-3.8,ymax=0, x=(i)+(timing[k]/2),lw=1,color="black",alpha=0.7,linestyles='dotted')
                            ax.annotate(s=timing_treatment[k],xy=(i+(timing[k]/2),-4.5), 
                                        ha='center',va='top')
                            i = i+timing[k]+21 # interval break
                    else:
                        # periods
                        rect = mpatch.Rectangle(xy=(i,0),width=timing[k],height=0.5,color=timing_colors[k],alpha=1., zorder=2)
                        ax.annotate(s=k,xy=(i+(timing[k]/2),0.23),ha='center',va='center',fontweight='bold')
                        ax.add_patch(rect)
                        # treatments
                        if k == 'C-1d':
                            ax.vlines(ymin=-3.8,ymax=0, x=i+(timing[k]/2),lw=1,color="black",alpha=0.7,linestyles='dotted')
                            ax.annotate(s=timing_treatment[k],xy=(i+(timing[k]/2),-4.5),ha='center',va='top')
                            i = i+timing[k]+21 # interval break
                        elif k == 'C-3d':
                            ax.vlines(ymin=-3.8,ymax=0, x=i+(timing[k]/2),lw=1,color="black",alpha=0.7,linestyles='dotted')
                            ax.annotate(s=timing_treatment[k],xy=(i+(timing[k]/2),-4.5),ha='center',va='top')
                            i = i+timing[k]+21 # interval break
                        else:
                            ax.vlines(ymin=-3.8,ymax=0, x=i+(timing[k]/2),lw=1,color="black",alpha=0.7,linestyles='dotted')
                            ax.annotate(s=timing_treatment[k],xy=(i+(timing[k]/2),-4.5), 
                                        ha='center',va='top')
                            i = i+timing[k]+21 # interval break
            else:
                if clinical_pat.loc[0,'Maintenance'] == 'Yes':
                
                    # period
                    rect = mpatch.Rectangle(xy=(i,0),width=timing[k],height=0.5,color=timing_colors[k],alpha=1., zorder=2)
                    ax.annotate(s=k,xy=(i+(timing[k]/2),0.23),ha='center',va='center',fontweight='bold')
                    ax.add_patch(rect)

                    # treatment
                    ax.vlines(ymin=-3.8,ymax=0, x=i+(timing[k]/2),lw=1,color="black",alpha=0.7,linestyles='dotted')
                    ax.annotate(s=timing_treatment[k],xy=(i+(timing[k]/2),-4.5), 
                                ha='center',va='top')

                    i = i+timing[k]

                else:
                    if clinical_pat.loc[0,'Transplant'] == 'Yes':

                        # treatment
                        ax.vlines(ymin=-3.8,ymax=0, x=i+(timing[k]/4),lw=1,color="black",alpha=0.7,linestyles='dotted')
                        ax.annotate(s="Alo-HSCT",xy=(i+(timing[k]/4),-4.5), ha='center',va='top')
    
    # Diagnosis date and blasts

    ax.scatter(x=-0.5, y=1, marker='o', color='black') 
    ax.vlines(ymin=0,ymax=1, x=-0.5,lw=1,color="black",alpha=1.)
    ax.text(s="Sample at diagnosis: \n {} %blasts".format(clinical_pat.loc[0,'Primary_sample_blasts']), 
            x=-0.5,y=1.2,rotation=45, va='bottom')

    ax.scatter(x=-0.5, y=-0.5, marker='o', color='black') 
    ax.vlines(ymin=-0.5,ymax=0, x=-0.5,lw=1,color="black",alpha=1.)
#     ax.text(s="Diagnosis\n{}".format(str(clinical_pat.loc[0,'Primary_biopsy_date'])), 
    ax.text(s="Diagnosis\n1",x=-0.5, y=-0.5,rotation=45, va='top', ha='right', 
           bbox=dict(fc="#f0f0f0", ec='#5b6b6a', pad=1, boxstyle="round,pad=0.1"))

    # Remission date and blasts
    
    if pd.isnull(clinical_pat.loc[0, 'days_between_pry_rem']) == False:
        ax.scatter(x=clinical_pat.loc[0,'days_between_pry_rem'], y=-0.5, marker='o', color='black') 
        ax.vlines(ymin=-0.5,ymax=0, x=clinical_pat.loc[0,'days_between_pry_rem'],lw=1,color="black",alpha=1.)
    #    ax.text(s="Remission\n{}".format(str(clinical_pat.loc[0,'Remission_biopsy_date'])),
        ax.text(s="Remission\n{}".format(str(int(clinical_pat.loc[0,'days_between_pry_rem']))),
                x=clinical_pat.loc[0,'days_between_pry_rem'],y=-0.5,rotation=45, 
                va='top', ha='right', 
               bbox=dict(fc="#f0f0f0", ec='#5b6b6a', pad=1, boxstyle="round,pad=0.1"))

        ax.scatter(x=clinical_pat.loc[0,'days_between_pry_rem'], y=1.2, marker='o', color='black') 
        ax.vlines(ymin=0,ymax=1.2, x=clinical_pat.loc[0,'days_between_pry_rem'],lw=1,color="black",alpha=1.)
        ax.text(s="Sample at remission \n {} %blasts".format(clinical_pat.loc[0,'Remission_sample_blasts']), 
                x=clinical_pat.loc[0,'days_between_pry_rem'],y=1.2,rotation=45, va='bottom')

    # Relapse date and blasts
    
    if pd.isnull(clinical_pat.loc[0,'days_between_pry_rel']) == False:
        ax.scatter(x=clinical_pat.loc[0,'days_between_pry_rel'], y=-0.5, marker='o', color='black') 
        ax.vlines(ymin=-0.5,ymax=0, x=clinical_pat.loc[0,'days_between_pry_rel'],lw=1,color="black",alpha=1.)
#        ax.text(s="Relapse\n{}".format(str(clinical_pat.loc[0,'Relapse_biopsy_date'])), 
        ax.text(s="Relapse\n{}".format(str(int(clinical_pat.loc[0,'days_between_pry_rel']))), 
                x=clinical_pat.loc[0,'days_between_pry_rel'],y=-0.5,rotation=45, 
                va='top', ha='right', 
                bbox=dict(fc="#f0f0f0", ec='#5b6b6a', pad=1, boxstyle="round,pad=0.1"))

        ax.scatter(x=clinical_pat.loc[0,'days_between_pry_rel'], y=1.2, marker='o', color='black') 
        ax.vlines(ymin=0,ymax=1.2, x=clinical_pat.loc[0,'days_between_pry_rel'],lw=1,color="black",alpha=1.)
        ax.text(s="Sample at relapse \n {} %blasts".format(clinical_pat.loc[0, "Relapse_sample_blasts"]), 
            x=clinical_pat.loc[0,'days_between_pry_rel'],y=1.2,rotation=45, va='bottom')

    # One year limit
    ax.scatter(x=365, y=-0.5, marker='o', color='black') 
    ax.vlines(ymin=-0.5,ymax=0, x=365,lw=1,color="black",alpha=1.)
    ax.text(s="One year\nfrom diagnosis", x=365,y=-0.5,rotation=45, va='top', ha='right')

    # other blast information

    for measure, days in cum_sum.items():
        if pd.isnull(clinical_pat.loc[0,measure]) == False:
            print(clinical_pat.loc[0,measure])
            ax.scatter(x=cum_sum[measure], y=1.2, marker='o', color='black') 
            ax.vlines(ymin=0,ymax=1.2, x=cum_sum[measure],lw=1,color="black",alpha=1.)
            ax.text(s="{} %blasts".format(clinical_pat.loc[0, measure]), 
                x=cum_sum[measure],y=1.2,rotation=45, va='bottom', ha='left')

    if clinical_pat.loc[0, 'Refractory'] == 'Yes':
        re_type = 'RFA'
    else:
        re_type = 'RC'
    ax.set_yticks([2,0,-2,-4,-6])
    ax.set_yticklabels(('',"{}\n{} {} yrs {}".format(pat, clinical_pat.loc[0,'Primary_immunoclassification'], 
                                        clinical_pat.loc[0,'Primary_diagnosis_age'], re_type),'',''),
                       fontweight='bold', ha='center')

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)

    plt.tick_params(
        axis='both',          
        which='both',      
        bottom=False,      
        top=False,
        right=False,
        left=False,
        labelbottom=False,
        pad=60)

    fig.savefig(os.path.join(out_path,"clinical_{}.svg".format(pat)), dpi=300,  bbox_inches='tight')
    plt.show()

In [ ]:
clinical['Treatment_protocol'].unique()